Pre-processing

In [7]:
!pip install "kagglehub[hf-datasets]"

In [29]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np

file_path_db1 = "olist_order_items_dataset.csv"

orderItems = kagglehub.load_dataset(
    KaggleDatasetAdapter.HUGGING_FACE,
    "olistbr/brazilian-ecommerce",
    file_path_db1,
)

print("Hugging Face Dataset:", orderItems)

file_path_db2 = "olist_products_dataset.csv"

items = kagglehub.load_dataset(
    KaggleDatasetAdapter.HUGGING_FACE,
    "olistbr/brazilian-ecommerce",
    file_path_db2,
)

print("Hugging Face Dataset:", items)

file_path_db3 = "olist_orders_dataset.csv"

order = kagglehub.load_dataset(
    KaggleDatasetAdapter.HUGGING_FACE,
    "olistbr/brazilian-ecommerce",
    file_path_db3,
)

print("Hugging Face Dataset:", order)

orderItems = orderItems.to_pandas()
items = items.to_pandas()
order = order.to_pandas()

df = orderItems.merge(items, on="product_id", how="left") #merge 1
df = df.merge(order[["order_id", "order_purchase_timestamp"]], on="order_id", how="left")#merge2

# converter timestamp
df["order_purchase_timestamp"] = pd.to_datetime(df["order_purchase_timestamp"])
df.head()


/tmp/ipython-input-2664359788.py:8: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  orderItems = kagglehub.load_dataset(


Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Hugging Face Dataset: Dataset({
    features: ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value'],
    num_rows: 112650
})


/tmp/ipython-input-2664359788.py:18: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  items = kagglehub.load_dataset(


Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Hugging Face Dataset: Dataset({
    features: ['product_id', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm'],
    num_rows: 32951
})


/tmp/ipython-input-2664359788.py:28: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  order = kagglehub.load_dataset(


Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Hugging Face Dataset: Dataset({
    features: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'],
    num_rows: 99441
})


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,order_purchase_timestamp
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,2017-09-13 08:59:02
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,pet_shop,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,2017-04-26 10:53:06
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,moveis_decoracao,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,2018-01-14 14:33:31
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,perfumaria,42.0,480.0,1.0,200.0,16.0,10.0,15.0,2018-08-08 10:00:35
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,ferramentas_jardim,59.0,409.0,1.0,3750.0,35.0,40.0,30.0,2017-02-04 13:57:51


Defining features


In [31]:
df["month"] = df["order_purchase_timestamp"].dt.to_period("M").dt.to_timestamp()

monthly = (
    df.groupby(["product_id", "month"])
      .agg(monthly_sales=("order_item_id", "count"))
      .reset_index()
)

monthly = monthly.merge( #atributos estatícos
    items[[
        "product_id",
        "product_category_name",
        "product_name_lenght",
        "product_description_lenght",
        "product_photos_qty",
        "product_weight_g",
        "product_length_cm",
        "product_height_cm",
        "product_width_cm"
    ]],
    on="product_id",
    how="left"
)

monthly.head() #assim temos as vendas por mês

,product_id,month,monthly_sales,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00066f42aeeb9f3007548bb9d3f33c38,2018-05-01,1,perfumaria,53.0,596.0,6.0,300.0,20.0,16.0,16.0
1,00088930e925c41fd95ebfe695fd2655,2017-12-01,1,automotivo,56.0,752.0,4.0,1225.0,55.0,10.0,26.0
2,0009406fd7479715e4bef61dd91f2462,2017-12-01,1,cama_mesa_banho,50.0,266.0,2.0,300.0,45.0,15.0,35.0
3,000b8f95fcb9e0096488278317764d19,2018-08-01,2,utilidades_domesticas,25.0,364.0,3.0,550.0,19.0,24.0,12.0
4,000d9be29b5207b54e86aa1b1ac54872,2018-04-01,1,relogios_presentes,48.0,613.0,4.0,250.0,22.0,11.0,15.0
